In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
import matplotlib.pyplot as plt
import math
import os

In [2]:
file_path = '/Users/ashwinravishankar/Work/WineQuality/Dataset/winequality.csv'
csv = pd.read_csv(file_path)
copy_csv = csv.copy()

print(list(csv))

csv.head()

['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
# tf.enable_eager_execution()

### See if anny rows have NA values
csv.isna()
csv.isna().sum()

### To drop rows that have empty feature values
# csv = csv.dropna()

### One hot encoding of categorical values
### View joint distribution of data



fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [4]:
type(csv)

pandas.core.frame.DataFrame

In [5]:
### Split train and test dataset
train = csv.sample(frac=0.8, random_state=0)
test = csv.drop(train.index)

In [6]:
### Statistics of training data

t_stats = train.describe()
t_stats.pop("quality")
t_stats = t_stats.transpose()
t_stats

,count,mean,std,min,25%,50%,75%,max
fixed acidity,1279.0,8.310164,1.744190,4.60000,7.1000,7.90000,9.250000,15.9000
volatile acidity,1279.0,0.525571,0.176645,0.12000,0.3900,0.52000,0.635000,1.5800
citric acid,1279.0,0.271618,0.196822,0.00000,0.0900,0.26000,0.430000,0.7900
residual sugar,1279.0,2.516341,1.300985,0.90000,1.9000,2.20000,2.600000,13.9000
chlorides,1279.0,0.087347,0.045666,0.01200,0.0710,0.07900,0.091000,0.6110
free sulfur dioxide,1279.0,15.868647,10.438880,1.00000,7.0000,14.00000,21.000000,72.0000
total sulfur dioxide,1279.0,46.488663,32.952841,6.00000,22.0000,38.00000,62.000000,289.0000
density,1279.0,0.996739,0.001888,0.99007,0.9956,0.99676,0.997855,1.0032
pH,1279.0,3.312588,0.153923,2.87000,3.2100,3.31000,3.400000,4.0100
sulphates,1279.0,0.655012,0.160192,0.33000,0.5500,0.62000,0.730000,1.9800


In [7]:
train_target = train.pop("quality")
test_target = test.pop("quality")

In [8]:
# train, test, train_target, test_target

In [9]:
### Normalize the data

def normalize(x):
    return (x - t_stats['mean']) / t_stats['std']

normal_train = normalize(train)
normal_test = normalize(test)

In [10]:
### Build model

model = tf.keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(train.keys())]),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(1)
])

model.compile(loss='mse',
              optimizer='sgd',
              metrics=[tf.keras.metrics.Accuracy()])


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                768       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 4,993
Trainable params: 4,993
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.predict(normal_train[:1])

array([[-0.3723266]], dtype=float32)

In [13]:
checkpoint_path = 'model/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)

In [14]:
total_train_records = len(normal_train)
total_test_records = len(train_target)
for j in range(3):
    for i in range(total_train_records):
        cb = model.fit(normal_train.iloc[[i]], train_target.iloc[[i]], batch_size=1, shuffle=False, callbacks=[cp_callback])

Instructions for updating:
Use tf.cast instead.

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
1/1 [==============================] - 0s 124ms/sample - loss: 40.6065 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 8ms/sample - loss: 24.6777 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 35.2312 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 19.0919 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consid

1/1 [==============================] - 0s 7ms/sample - loss: 8.7986 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 2.7944 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 0.1164 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 2.4481 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 8ms/sample - loss: 0.0566 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 8ms

1/1 [==============================] - 0s 7ms/sample - loss: 12.2854 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 19.2428 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 22.3770 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 8ms/sample - loss: 16.6863 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 38.3931 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0

1/1 [==============================] - 0s 9ms/sample - loss: 6.3786 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 9ms/sample - loss: 9.5617 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 9ms/sample - loss: 2.1404 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 8ms/sample - loss: 0.5008 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 8ms/sample - loss: 13.2555 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 6m

1/1 [==============================] - 0s 8ms/sample - loss: 1.8830 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 1.2707 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 8ms/sample - loss: 0.4508 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 3.0714 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 8ms/sample - loss: 0.2238 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 8ms

...

1/1 [==============================] - 0s 7ms/sample - loss: 0.3845 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 6ms/sample - loss: 0.0761 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 6ms/sample - loss: 0.0069 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 6ms/sample - loss: 0.0798 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 6ms/sample - loss: 0.7995 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 6ms

1/1 [==============================] - 0s 6ms/sample - loss: 0.2852 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 0.0570 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 10ms/sample - loss: 0.1243 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 1.4991 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 6ms/sample - loss: 0.0710 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7m

1/1 [==============================] - 0s 6ms/sample - loss: 1.9872e-04 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 0.1179 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 0.0618 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 6ms/sample - loss: 0.1796 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 0.0309 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s

1/1 [==============================] - 0s 7ms/sample - loss: 0.1845 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 6ms/sample - loss: 0.0098 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 0.2825 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 0.8379 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 0.0739 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms

1/1 [==============================] - 0s 6ms/sample - loss: 0.0124 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 0.4420 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 0.1825 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 1.1244 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 7ms/sample - loss: 1.7715 - accuracy: 0.0000e+00

Epoch 00001: saving model to model/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1/1 [==============================] - 0s 6ms

In [ ]:
# normal_train.iloc[0].values.shape

In [ ]:
hist = pd.DataFrame(cb.history)
# hist['epoch'] = cb.epoch
hist.head()

In [15]:
model.save('model/reg.h5')

In [ ]:
# def plot_history(history):
#   hist = pd.DataFrame(history.history)
#   hist['epoch'] = history.epoch
  
#   plt.figure()
#   plt.xlabel('Epoch')
#   plt.ylabel('Mean Abs Error [MPG]')
#   plt.plot(hist['epoch'], hist['mean_absolute_error'],
#            label='Train Error')
#   plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
#            label = 'Val Error')
# #   plt.ylim([0,5])
#   plt.legend()
  
#   plt.figure()
#   plt.xlabel('Epoch')
#   plt.ylabel('Mean Square Error [$MPG^2$]')
#   plt.plot(hist['epoch'], hist['mean_squared_error'],
#            label='Train Error')
#   plt.plot(hist['epoch'], hist['val_mean_squared_error'],
#            label = 'Val Error')
# #   plt.ylim([0,20])
#   plt.legend()
#   plt.show()


# plot_history(cb)

In [ ]:
print(test_target.iloc[1], model.predict(normal_test[1:2])[0])

# test_pred = model.predict(normal_test).flatten()

In [ ]:
eval = model.evaluate(normal_test, test_target)

# acc = tf.keras.metrics.Accuracy()
# acc.update_state(test_target, test_pred)
# print(acc.result().numpy())

In [ ]:
for i in range(len(test_target)):
    print(test_target.iloc[i], '\t', int(round(test_pred[i])))

In [ ]:
# test_predictions = model.predict(normal_test).flatten()

# plt.scatter(test_target, np.rint(test_predictions))
# plt.xlabel('True Values [MPG]')
# plt.ylabel('Predictions [MPG]')
# plt.axis('equal')
# plt.axis('square')
# plt.xlim([0,plt.xlim()[1]])
# plt.ylim([0,plt.ylim()[1]])
# _ = plt.plot([-100, 100], [-100, 100])

In [ ]:
# test_target_2 = test_target.copy()
# test_target_2 = test_target_2.values
test_target_2

In [ ]:
label = tf.convert_to_tensor(test_target_2)
pred = tf.convert_to_tensor(test_pred)
acc,acc_op = tf.metrics.accuracy(label, pred)

sess = tf.Session()

sess.run(tf.local_variables_initializer())
sess.run(tf.global_variables_initializer())

print(sess.run([acc, acc_op]))
# sess.close()
# acc.update_state(test_target, test_pred)
# print(acc.result().numpy())

In [ ]:
test_target

In [ ]:
path = '/Users/ashwinravishankar/Work/WineQuality/JupyterNotebook/model'
tf.keras.models.save_model(model, path)